In [1]:
import copy
import numpy as np
import pandas as pd

In [2]:
data_pre = pd.read_excel('./data.xlsx')

In [3]:
data_pre

,方案,P1,P2,P3,P4
0,方案1,3.0,1.0,1.0,70
1,方案2,2.5,0.8,0.8,50
2,方案3,1.8,0.5,2.0,110
3,方案4,2.2,0.7,1.2,90


In [4]:
labels = data_pre.keys()[1:]
names = data_pre['方案']
values = data_pre[labels].values

In [5]:
for i in [0, 2]:
    values[:, i] = (np.max(values[:, i]) - values[:, i])/(np.max(values[:, i]) - np.min(values[:, i]))

In [6]:
for i in [1, 3]:
    values[:, i] = (values[:, i] - np.min(values[:, i]))/(np.max(values[:, i]) - np.min(values[:, i]))

In [7]:
values

array([[0.        , 1.        , 0.83333333, 0.33333333],
       [0.41666667, 0.6       , 1.        , 0.        ],
       [1.        , 0.        , 0.        , 1.        ],
       [0.66666667, 0.4       , 0.66666667, 0.66666667]])

In [8]:
# 组合权重
#各个赋权方法得到的权重
W1=[0.3,0.4,0.15,0.15]
W2=[0.4,0.3,0.15,0.15]
W3=[0.25,0.25,0.25,0.25]
W4=[0.2403,0.2294,0.3062,0.2242]
W=np.array([W1,W2,W3,W4])
r=np.corrcoef(W)   #求皮尔逊相关系数
r = np.nan_to_num(r)
# pearson相关系数 显著性检验
t = np.abs(r) * np.sqrt((len(r) - 2)/(1 - r ** 2))
t[np.isinf(t)] = 0
alpha = 0.05
# 求解 t 分布 alpha=0.05 上分位数
from scipy import stats
t_num = stats.t.isf(alpha/2, df=len(r)-2)

D:\Anaconda3\envs\ai_py\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
D:\Anaconda3\envs\ai_py\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
D:\Anaconda3\envs\ai_py\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [9]:
t

array([[0.        , 1.75      , 0.        , 0.75965385],
       [1.75      , 0.        , 0.        , 0.60848883],
       [0.        , 0.        , 0.        , 0.        ],
       [0.75965385, 0.60848883, 0.        , 0.        ]])

In [10]:
# 粗暴求和, 比较矩阵的大小
if np.sum(t) < t_num * len(r) ** 2:
    print('一致性检验不通过')
    # CRITIC法赋权
    thegma = W.std(axis=1)
    f = np.sum((1-r), axis=1)
    c = thegma * f
    theta = c / c.sum()
    Wc = W.T * theta
    Wc = np.sum(Wc, axis=1)
else:
    print('一致性检验通过')
    Wc = np.sum(W, axis=0)
    Wc = Wc/np.sum(Wc)

一致性检验不通过


In [11]:
Wc

array([0.32916576, 0.32835201, 0.17881339, 0.16368728])

In [12]:
# 计算得分
s = values @ Wc
Score=100*s/max(s) #计算得分
for i in range(0,len(Score)):
    print(f"{data_pre['方案'][i]}百分制评分为：{Score[i]}") 

方案1百分制评分为：91.85091826552814
方案2百分制评分为：88.57894552940891
方案3百分制评分为：85.1040171990309
方案4百分制评分为：100.0
